In [1]:
import os
path = './'
os.makedirs(os.path.join(path, 'DataRes_Research_Assessment', 'data'), exist_ok=True)
root_dir = os.path.join(path, 'DataRes_Research_Assessment')


!pip3 install --upgrade gdown --quiet
!gdown 1z3B1GR7UtHZGrqNjUaep6thHwrN3IYSI

import tarfile
from tqdm import tqdm

tar = tarfile.open("data.tar.gz", "r:gz")
total_size = sum(f.size for f in tar.getmembers())
with tqdm(total=total_size, unit="B", unit_scale=True, desc="Extracting tar.gz file") as pbar:
    for member in tar.getmembers():
        tar.extract(member, os.path.join(root_dir, 'data'))
        pbar.update(member.size)
tar.close()

import urllib.request

import os
import pickle

import numpy as np

def load_subcategories(txt_path):
    subcategories = {}
    file = open(txt_path, 'r')
    lines = file.readlines()
    for i, l in enumerate(lines):
        info = l.split()
        info[0] = info[0][3:]
        subcategories.update({info[0]: {'ori_class_id': int(info[1]), 'class_id': i}})

    return subcategories


from tqdm import trange
import cv2
import random

def select_samples(subcategories, root_dir, split, n_images_per_subcategory):
    samples = []
    if split == "train":
        train_dir = os.path.join(root_dir, "data", "images", "train")
        for i in subcategories:
            child_dir = os.path.join(train_dir, i[0], i)
            pics = random.sample(os.listdir(child_dir), n_images_per_subcategory)
            for j in pics:
                samples.append((cv2.resize(cv2.imread(os.path.join(child_dir, j)), (32,32)).flatten().tolist(), subcategories[i]["class_id"]))
    elif split == "val":
        val_dir = os.path.join(root_dir, "data", "images")
        file = open(os.path.join(root_dir, "data", "val.txt"), 'r')
        lines = file.readlines()
        val_data = []
        for i in lines:
            val_data.append(i.split())
        random.shuffle(val_data)
        for i in subcategories:
            old_id = subcategories[i]["ori_class_id"]
            count = 0
            for j in val_data:
                if int(j[1]) == old_id:
                    samples.append((cv2.resize(cv2.imread(os.path.join(val_dir, j[0])), (32,32)).flatten().tolist(), subcategories[i]["class_id"]))
                    count += 1
                if count >= n_images_per_subcategory:
                    break
    return samples

def create_tinyplaces(samples, binary=True):
    data, labels = [], []
    for i in samples:
        data.append(i[0])
        if binary:
            if i[1] >= 10:
                labels.append(1)
            else:
                labels.append(0)
        else:
            labels.append(i[1])
    data = np.array(data)
    labels = np.array(labels)
    dataset = {"data": data, "label": labels}

    return dataset


# Set the root directory of the dataset
root_dir = './DataRes_Research_Assessment/data'

# Load the target subcategories and their class IDs
subcategories = load_subcategories(os.path.join(root_dir, 'data', 'categories_tinyplaces.txt'))

# Select the samples from the train split of the TinyPlaces dataset
train_samples = select_samples(subcategories, root_dir, 'train', 500)

# Create the TinyPlaces datasets for binary and multiclass classification
tinyplaces_binary_train = create_tinyplaces(train_samples, binary=True)
tinyplaces_multi_train = create_tinyplaces(train_samples, binary=False)

# Select the samples from the val split of the MiniPlaces dataset
val_samples = select_samples(subcategories, root_dir, 'val', 50)

# Create the TinyPlaces datasets for binary and multiclass classification
tinyplaces_binary_val = create_tinyplaces(val_samples, binary=True)
tinyplaces_multi_val = create_tinyplaces(val_samples, binary=False)

# Save the TinyPlaces datasets to the data directory
data_dir = os.path.join(root_dir, 'data')


with open(os.path.join(data_dir, 'tinyplaces_binary_train.pkl'), 'wb') as f:
    pickle.dump(tinyplaces_binary_train, f)

with open(os.path.join(data_dir, 'tinyplaces_multi_train.pkl'), 'wb') as f:
    pickle.dump(tinyplaces_multi_train, f)

with open(os.path.join(data_dir, 'tinyplaces_binary_val.pkl'), 'wb') as f:
    pickle.dump(tinyplaces_binary_val, f)

with open(os.path.join(data_dir, 'tinyplaces_multi_val.pkl'), 'wb') as f:
    pickle.dump(tinyplaces_multi_val, f)


class TinyPlacesDataset(object):
    def __init__(self, data_dict):
        self.dataset = data_dict
        self.num_samples = len(data_dict['data'])

    def subsample(self, ratio=0.1, seed=None):
        if seed is not None:
            np.random.seed(seed)

        nums = random.sample(range(self.num_samples), int(ratio * self.num_samples))
        sub_dataset = {'data': self.dataset['data'][nums], 'label': self.dataset['label'][nums]}
        subsampled_dataset = TinyPlacesDataset(sub_dataset)

        return subsampled_dataset

Downloading...
From (original): https://drive.google.com/uc?id=1z3B1GR7UtHZGrqNjUaep6thHwrN3IYSI
From (redirected): https://drive.google.com/uc?id=1z3B1GR7UtHZGrqNjUaep6thHwrN3IYSI&confirm=t&uuid=ea50ff38-a7b4-4200-bb5f-8e3409252d98
To: /kaggle/working/data.tar.gz
100%|█████████████████████████████████████████| 423M/423M [00:03<00:00, 121MB/s]


Extracting tar.gz file: 100%|██████████| 526M/526M [00:13<00:00, 38.8MB/s] 


In [2]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if device == torch.device('cuda'):
    print(f'Using device: {device}. Good to go!')
else:
    print('Please set GPU via Edit -> Notebook Settings.')
    
with open(os.path.join(root_dir, 'data', 'tinyplaces_binary_train.pkl'), 'rb') as f:
    binary_train = TinyPlacesDataset(pickle.load(f))
with open(os.path.join(root_dir, 'data', 'tinyplaces_binary_val.pkl'), 'rb') as f:
    binary_val = TinyPlacesDataset(pickle.load(f))
with open(os.path.join(root_dir, 'data', 'tinyplaces_multi_train.pkl'), 'rb') as f:
    multi_train = TinyPlacesDataset(pickle.load(f))
with open(os.path.join(root_dir, 'data', 'tinyplaces_multi_val.pkl'), 'rb') as f:
    multi_val = TinyPlacesDataset(pickle.load(f))

# Convert everything from numpy arrays to tensors and move them to the GPU using .cuda()
for dataset in [binary_train, binary_val, multi_train, multi_val]:
    for k in ['data', 'label']:
        dataset.dataset[k] = torch.tensor(dataset.dataset[k]).float().cuda()
        
from tqdm import tqdm

from torch.utils.data import Dataset
from PIL import Image

class MiniPlaces(Dataset):
    def __init__(self, root_dir, split, transform=None, label_dict=None):
        assert split in ['train', 'val', 'test']
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.filenames = []
        self.labels = []
        self.label_dict = label_dict if label_dict is not None else {}
        if split == "train" or split == "val":
            with open(os.path.join(root_dir, ("train" if self.split == "train" else "val") + ".txt")) as f:
                for line in f:
                    line = line.rstrip().split()
                    n = int(line[0][-12:-4])
                    if n <= 900:
                        self.filenames.append(os.path.join(line[0]))
                        self.labels.append(int(line[1]))
        if label_dict is None and split == "train":
            with open(os.path.join(root_dir, "train.txt")) as f:
                num = -1
                for line in f:
                    line = line.rstrip().split()
                    if int(line[1]) > num:
                        num += 1
                        self.label_dict.update({int(line[1]): line[0][8:line[0].find("/", 8)]})
        if split == "test":
            self.labels = os.listdir(os.path.join(root_dir, "images", "test"))
            self.filenames = ["test/" + i for i in self.labels]

    def __len__(self):
        dataset_len = len(self.labels)
        return dataset_len

    def __getitem__(self, idx):
        image = Image.open(os.path.join(self.root_dir, "images", self.filenames[idx]))
        if not self.transform is None:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

from torchvision import transforms

data_transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

data_root = os.path.join(root_dir, 'data')
miniplaces_train = MiniPlaces(data_root, split='train', transform=data_transform)
miniplaces_val = MiniPlaces(
    data_root, split='val',
    transform=data_transform,
    label_dict=miniplaces_train.label_dict)

def train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs):
    # Place model on device
    model = model.to(device)
    best_acc = 0
    flag = False
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        # Use tqdm to display a progress bar during training
        with tqdm(total=len(train_loader), desc=f'Epoch {epoch + 1}/{num_epochs}') as pbar:
            for inputs, labels in train_loader:
                # Move inputs and labels to device
                inputs = inputs.to(device)
                labels = labels.to(device)
                # Zero out gradients
                optimizer.zero_grad()
                # Compute the logits and loss
                logits = model(inputs)
                loss = criterion(logits, labels)
                # Backpropagate the loss
                loss.backward()
                # Update the weights
                optimizer.step()
                # Update the progress bar
                pbar.update(1)
                pbar.set_postfix(loss=loss.item())

        # Evaluate the model on the validation set
        avg_loss, accuracy = evaluate(model, val_loader, criterion, device)
        if best_acc > accuracy:
            if flag:
                print(f'Validation set: Average loss = {avg_loss:.4f}, Accuracy = {accuracy:.4f}')
                break
            else:
                flag = True
        else:
            best_acc = accuracy
            flag = False
        print(f'Validation set: Average loss = {avg_loss:.4f}, Accuracy = {accuracy:.4f}')

def evaluate(model, test_loader, criterion, device):
    """
    Evaluate the classifier on the test set.

    Args:
        model: classifier to evaluate.
        test_loader (torch.utils.data.DataLoader): Data loader for the test set.
        criterion (callable): Loss function to use for evaluation.
        device (torch.device): Device to use for evaluation.

    Returns:
        float: Average loss on the test set.
        float: Accuracy on the test set.
    """
    model.eval()  # Set model to evaluation mode

    with torch.no_grad():
        total_loss = 0.0
        num_correct = 0
        num_samples = 0

        for inputs, labels in test_loader:
            # Move inputs and labels to device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Compute the logits and loss
            logits = model(inputs)
            loss = criterion(logits, labels)
            total_loss += loss.item()

            # Compute the accuracy
            _, predictions = torch.max(logits, dim=1)
            num_correct += (predictions == labels).sum().item()
            num_samples += len(inputs)

    # Compute the average loss and accuracy
    avg_loss = total_loss / len(test_loader)
    accuracy = num_correct / num_samples

    return avg_loss, accuracy

def predict(model, test_dataloader):
    """
    Evaluate the classifier on the test set.

    Args:
        model: classifier to evaluate.
        test_loader (torch.utils.data.DataLoader): Data loader for the test set.
        criterion (callable): Loss function to use for evaluation.
        device (torch.device): Device to use for evaluation.

    Returns:
        float: Average loss on the test set.
        float: Accuracy on the test set.
    """
    out = []
    for i in test_dataloader:
        pic = i[0]
        lab = torch.argmax(model.to('cpu')(pic))
        out.append(lab.item())

    return out

data_transform_flatten = transforms.Compose([data_transform, torch.flatten])

Using device: cuda. Good to go!


In [3]:
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import torchvision.transforms as v1
from torchvision.io import read_image


class resBlock(nn.Module):
    def __init__(
        self,
        channels,
        input_size=(64,64),
        dropout_rate=0.3,
        kernel_size=3, stride=1, padding=1):

        super().__init__()

        self.process = nn.Sequential (
            nn.Conv2d(channels, channels, kernel_size=kernel_size, stride=stride, padding=padding),
            nn.MaxPool2d(3, stride=1, padding = 1),
            nn.Dropout(p=dropout_rate),
            nn.Conv2d(channels, channels, kernel_size=kernel_size, padding=padding),
            nn.MaxPool2d(3, stride=1, padding = 1)
        )


    def forward(self, x):
        left = self.process(x)
        right = x

        return (left + right)



class SlowConv(nn.Module):
    def __init__(
        self,
        input_channels, conv_hidden_channels, conv_out_channels,
        input_size=(64,64),
        dropout_rate=0.25,
        fc_out_channels=128, fc_hidden_layer = 256, num_classes=100,
        kernel_size=3, stride=1, padding=1):

      super().__init__()

      self.preprocess = nn.Sequential (
            nn.Conv2d(input_channels, conv_hidden_channels, kernel_size = 3, stride=stride, padding = padding),
            nn.ReLU(inplace=True),
            nn.Conv2d(conv_hidden_channels, conv_out_channels, kernel_size = kernel_size, padding = padding),
            nn.MaxPool2d(kernel_size=kernel_size, stride=1, padding = 1),
            nn.Dropout(p=dropout_rate),
        )


      self.cnn = nn.Sequential (
            resBlock(conv_out_channels, conv_out_channels, stride=stride, dropout_rate=dropout_rate),
            resBlock(conv_out_channels, conv_out_channels, stride=stride, dropout_rate=dropout_rate),
            resBlock(conv_out_channels, conv_out_channels, stride=stride, dropout_rate=dropout_rate),
        )

      self.linear = nn.Sequential (

            nn.Linear(262144*2, fc_hidden_layer),
            nn.ELU(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(fc_hidden_layer, fc_hidden_layer),
            nn.ELU(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(fc_hidden_layer, int(fc_hidden_layer/2)),
            nn.ELU(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(int(fc_hidden_layer/2), num_classes),
        )


    def forward(self, x):

        x = self.preprocess(x)

        x = self.cnn(x)
        x = x.flatten(1)
        x = self.linear(x)

        return x


In [4]:
def initialize_weights(m):
    if isinstance(m, nn.Conv1d):
        nn.init.kaiming_uniform_(m.weight.data,nonlinearity='relu')
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0)

    elif isinstance(m, nn.BatchNorm1d):
        nn.init.constant_(m.weight.data, 1)
        nn.init.constant_(m.bias.data, 0)

    elif isinstance(m, nn.Linear):
        nn.init.kaiming_uniform_(m.weight.data)
        nn.init.constant_(m.bias.data, 0)

In [7]:
### Create your DataLoader and use predict to get your model's predictions
conv_train_dataset = MiniPlaces(
    root_dir=data_root, split='train',
    transform=data_transform)
conv_val_dataset = MiniPlaces(
    root_dir=data_root, split='val',
    transform=data_transform)
conv_train_loader = torch.utils.data.DataLoader(
    conv_train_dataset, batch_size=64, num_workers=0, shuffle=True)
conv_val_loader = torch.utils.data.DataLoader(
    conv_val_dataset, batch_size=64, num_workers=0, shuffle=False)

model = SlowConv(3,64,128,dropout_rate=0.3, fc_hidden_layer=1024)
# model = model.apply(initialize_weights)
optimizer = torch.optim.SGD(model.parameters(), momentum=0.8, weight_decay=0.001, lr=0.01)
criterion = nn.CrossEntropyLoss()

In [27]:
! rm model-epoch1*.pth

In [14]:
#After 15 epoch, loss is small

optimizer = torch.optim.SGD(model.parameters(), momentum=0.1, weight_decay=0.001, lr=0.001)

In [ ]:
for epoch in range(32,100,3):
    train(model, conv_train_loader, conv_val_loader, optimizer, criterion, device, num_epochs=3)
    torch.save(model, 'model-epoch'+str(epoch)+'.pth')

Epoch 1/3: 100%|██████████| 1407/1407 [07:42<00:00,  3.04it/s, loss=0.00427]


Validation set: Average loss = 4.7443, Accuracy = 0.2333


Epoch 2/3: 100%|██████████| 1407/1407 [07:43<00:00,  3.03it/s, loss=0.0159] 


Validation set: Average loss = 4.7288, Accuracy = 0.2322


Epoch 3/3: 100%|██████████| 1407/1407 [07:44<00:00,  3.03it/s, loss=0.00261]


...

Epoch 3/3: 100%|██████████| 1407/1407 [07:41<00:00,  3.05it/s, loss=0.00943]


Validation set: Average loss = 4.6210, Accuracy = 0.2322


Epoch 1/3: 100%|██████████| 1407/1407 [07:41<00:00,  3.05it/s, loss=0.00777]


Validation set: Average loss = 4.6153, Accuracy = 0.2322


Epoch 2/3:  31%|███       | 438/1407 [02:23<05:19,  3.04it/s, loss=0.00732]